In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define paths
processed_train_path = '/root/autodl-tmp/projects/SL_NSL/dataset/processed/KDDTrain_processed.csv'
processed_test_path = '/root/autodl-tmp/projects/SL_NSL/dataset/processed/KDDTest_processed.csv'

In [2]:
# Load training data
print("Loading processed training data...")
df_train = pd.read_csv(processed_train_path)

# Select features and labels
X = df_train.drop(columns=['label', 'binary_label'])
y_binary = df_train['binary_label']

# Display data info
print("Feature matrix shape:", X.shape)
print("Label distribution:")
print(y_binary.value_counts())

Loading processed training data...


Feature matrix shape: (125973, 108)
Label distribution:
0    67343
1    58630
Name: binary_label, dtype: int64


In [3]:
# Split into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.2, random_state=42
)

print("Dataset split complete")
print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_test.shape[0]}")

Dataset split complete
Training samples: 100778
Validation samples: 25195


In [4]:
# Create and train XGBoost model
print("\nTraining XGBoost model...")
rf_binary = XGBClassifier(n_estimators=50, random_state=42)
rf_binary.fit(X_train, y_train)

# Display feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_binary.feature_importances_
})
print("\nTop 10 most important features:")
display(feature_importance.sort_values('importance', ascending=False).head(10))


Training XGBoost model...



Top 10 most important features:


,feature,importance
16,service_ecr_i,0.357669
96,src_bytes,0.213946
5,service_http,0.103912
13,diff_srv_rate,0.050648
2,logged_in,0.027883
19,wrong_fragment,0.027107
101,dst_bytes,0.022643
71,is_guest_login,0.019332
6,service_private,0.016268
20,dst_host_same_src_port_rate,0.015376


In [5]:
# Predict and evaluate on validation set
print("\nEvaluating on validation set...")
y_pred = rf_binary.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Validation Accuracy: {accuracy:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


Evaluating on validation set...
Validation Accuracy: 1.00

Confusion Matrix:
[[13407    15]
 [   16 11757]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13422
           1       1.00      1.00      1.00     11773

    accuracy                           1.00     25195
   macro avg       1.00      1.00      1.00     25195
weighted avg       1.00      1.00      1.00     25195



In [6]:
# Load and evaluate on external test set
print("\nEvaluating on external test set...")
df_test = pd.read_csv(processed_test_path)

X_external_test = df_test.drop(columns=['label', 'binary_label'])
y_external_test = df_test['binary_label']

# Make predictions
y_external_pred = rf_binary.predict(X_external_test)

# Calculate metrics
accuracy_external = accuracy_score(y_external_test, y_external_pred)
conf_matrix_external = confusion_matrix(y_external_test, y_external_pred)
class_report_external = classification_report(y_external_test, y_external_pred)

print(f"External Test Set Accuracy: {accuracy_external:.2f}")
print("\nConfusion Matrix:")
print(conf_matrix_external)
print("\nClassification Report:")
print(class_report_external)


Evaluating on external test set...
External Test Set Accuracy: 0.86

Confusion Matrix:
[[ 6813  2898]
 [  330 12503]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.70      0.81      9711
           1       0.81      0.97      0.89     12833

    accuracy                           0.86     22544
   macro avg       0.88      0.84      0.85     22544
weighted avg       0.87      0.86      0.85     22544

